# Subtraction_rnn

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


## Data Representation

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
            
        #print(x)
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789- '
ctable = CharacterTable(chars)
print(ctable)

## Data Generation

In [5]:
def Generate_Data(DIGITS, DATA_SIZE, MAXLEN):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        if(a >= b):
            key = tuple(sorted((a, b)))
            if key in seen:
                continue
            seen.add(key)
            # Pad the data with spaces such that it is always MAXLEN.
            q = '{}-{}'.format(a, b)
            query = q + ' ' * (MAXLEN - len(q))
            ans = str(a - b)
            # Answers can be of maximum size DIGITS + 1.
            ans += ' ' * (DIGITS + 1 - len(ans))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
                # space used for padding.)
                query = query[::-1]
            questions.append(query)
            expected.append(ans)
    print('Total addition questions:', len(questions))
    
    return questions, expected

## Feature Engineering

In [6]:
def Vectorization(questions, expected):
    print('Vectorization...')
    x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(expected):
        y[i] = ctable.encode(sentence, DIGITS + 1)

    return x, y

## Get Training Data and Validation Data
* total data = 50000, 45000 for training, 5000 for validation

In [7]:
questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
#print(questions[:10])
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## Build Model

In [8]:
def Build_Model(RNN, HIDDEN_SIZE, BATCH_SIZE, LAYERS, MAXLEN, DIGITS, chars):
    print('Build model...')
    model = Sequential()
    # "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
    # Note: In a situation where your input sequences have a variable length,
    # use input_shape=(None, num_feature).
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    # As the decoder RNN's input, repeatedly provide with the last output of
    # RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
    # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
    model.add(layers.RepeatVector(DIGITS + 1))
    # The decoder RNN could be multiple layers stacked or a single layer.
    for _ in range(LAYERS):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))

    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    
    return model

In [9]:
def Train_Step(model, epoch, REVERSE = True):
    for iteration in range(1, epoch):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(x_val, y_val))
        # Select 10 samples from the validation set at random so we can visualize
        # errors.
        if iteration == epoch-1:
            for i in range(10):
                ind = np.random.randint(0, len(x_val))
                rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
                preds = model.predict_classes(rowx, verbose=0)
                q = ctable.decode(rowx[0])
                correct = ctable.decode(rowy[0])
                guess = ctable.decode(preds[0], calc_argmax=False)
                print('Q', q[::-1] if REVERSE else q, end=' ')
                print('T', correct, end=' ')
                if correct == guess:
                    print(colors.ok + '☑' + colors.close, end=' ')
                else:
                    print(colors.fail + '☒' + colors.close, end=' ')
                print(guess)

## String Matching
* training 100 epochs

In [10]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [11]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 221us/step - loss: 1.7508 - acc: 0.3608 - val_loss: 1.6105 - val_acc: 0.4114

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 167us/step - loss: 1.5012 - acc: 0.4491 - val_loss: 1.4034 - val_acc: 0.4727

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 171us/step - loss: 1.3390 - acc: 0.5032 - val_loss: 1.2519 - val_acc: 0.5368

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 168us/step - loss: 1.1063 - acc: 0.6119 - val_loss: 1.0154 - val_acc: 0.6496

45000/45000 [==============================] - 7s 163us/step - loss: 0.0081 - acc: 0.9993 - val_loss: 0.0105 - val_acc: 0.9987

--------------------------------------------------
Iteration 34
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 0.0076 - acc: 0.9993 - val_loss: 0.0107 - val_acc: 0.9983

--------------------------------------------------
Iteration 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 0.0351 - acc: 0.9896 - val_loss: 0.0403 - val_acc: 0.9873

--------------------------------------------------
Iteration 36
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 0.0127 - acc: 0.9974 - val_loss: 0.0102 - val_acc: 0.9982

--------------------------------------------------
Iteration 37
Train on 45000 samples, validate on 5000 samples
Epoch 

45000/45000 [==============================] - 7s 162us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0031 - val_acc: 0.9993

--------------------------------------------------
Iteration 67
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0030 - val_acc: 0.9994

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 0.9993

--------------------------------------------------
Iteration 69
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 9.2917e-04 - acc: 1.0000 - val_loss: 0.0026 - val_acc: 0.9994

--------------------------------------------------
Iteration 70
Train on 45000 samples, validate on 5000 samples
Ep

45000/45000 [==============================] - 7s 163us/step - loss: 3.9040e-04 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 0.9996

--------------------------------------------------
Iteration 99
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 3.6043e-04 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9995
Q 575-266 T 309  ☑ 309 
Q 475-305 T 170  ☑ 170 
Q 146-104 T 42   ☑ 42  
Q 482-64  T 418  ☑ 418 
Q 459-43  T 416  ☑ 416 
Q 43-38   T 5    ☑ 5   
Q 917-90  T 827  ☑ 827 
Q 738-1   T 737  ☑ 737 
Q 926-38  T 888  ☑ 888 
Q 552-1   T 551  ☑ 551 


## Testing

In [ ]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

In [ ]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

## Result
### * 利用LSTM做encoder-decoder的seq2seq架構來實現減法器
### * Training data總數為50000筆，45000為training，5000為validation
### * Batch size = 128
### * RNN hidden layer = 128
### * Optimizer使用adam
### * Epochs = 100
### * Validation result可達99.97%
### * Testing data總數為1000筆，Testing accuracy可達 99.8%

## Other Discussion
* 實驗使用不同的epoch和batch size訓練
* 實驗不同位數的數字
* 實驗"more number subtract"

### 1.1 實驗使用不同的epoch和batch size訓練
* with batch = 64, epoch = 100

In [14]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 64
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [15]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 355us/step - loss: 1.6513 - acc: 0.3974 - val_loss: 1.4330 - val_acc: 0.4658

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 326us/step - loss: 1.2950 - acc: 0.5279 - val_loss: 1.1223 - val_acc: 0.5919

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 328us/step - loss: 0.9842 - acc: 0.6533 - val_loss: 0.8970 - val_acc: 0.6855

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 331us/step - loss: 0.7872 - acc: 0.7231 - val_loss: 0.6897 - val_acc: 0.7

45000/45000 [==============================] - 15s 324us/step - loss: 0.0029 - acc: 0.9997 - val_loss: 0.1185 - val_acc: 0.9692

--------------------------------------------------
Iteration 34
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 321us/step - loss: 0.0240 - acc: 0.9937 - val_loss: 0.0059 - val_acc: 0.9987

--------------------------------------------------
Iteration 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 319us/step - loss: 0.0023 - acc: 0.9999 - val_loss: 0.0036 - val_acc: 0.9994

--------------------------------------------------
Iteration 36
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 320us/step - loss: 0.0234 - acc: 0.9933 - val_loss: 0.0158 - val_acc: 0.9955

--------------------------------------------------
Iteration 37
Train on 45000 samples, validate on 5000 samples
Ep

45000/45000 [==============================] - 14s 322us/step - loss: 7.6332e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9998

--------------------------------------------------
Iteration 66
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 320us/step - loss: 5.6415e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9998

--------------------------------------------------
Iteration 67
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 321us/step - loss: 0.0142 - acc: 0.9962 - val_loss: 0.0978 - val_acc: 0.9715

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 321us/step - loss: 0.0086 - acc: 0.9976 - val_loss: 0.0054 - val_acc: 0.9987

--------------------------------------------------
Iteration 69
Train on 45000 samples, validate on 5000 sa

45000/45000 [==============================] - 14s 320us/step - loss: 0.0114 - acc: 0.9975 - val_loss: 0.2035 - val_acc: 0.9474

--------------------------------------------------
Iteration 98
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 317us/step - loss: 0.0145 - acc: 0.9958 - val_loss: 0.0033 - val_acc: 0.9992

--------------------------------------------------
Iteration 99
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 14s 318us/step - loss: 4.7923e-04 - acc: 1.0000 - val_loss: 0.0011 - val_acc: 0.9999
Q 674-5   T 669  ☑ 669 
Q 702-2   T 700  ☑ 700 
Q 979-173 T 806  ☑ 806 
Q 995-27  T 968  ☑ 968 
Q 273-54  T 219  ☑ 219 
Q 703-11  T 692  ☑ 692 
Q 222-9   T 213  ☑ 213 
Q 937-294 T 643  ☑ 643 
Q 820-63  T 757  ☑ 757 
Q 869-3   T 866  ☑ 866 


In [16]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 224-3   T 221  ☑ 221 
Q 982-266 T 716  ☑ 716 
Q 709-370 T 339  ☑ 339 
Q 577-6   T 571  ☑ 571 
Q 8-5     T 3    ☑ 3   
Q 48-7    T 41   ☑ 41  
Q 39-3    T 36   ☑ 36  
Q 441-6   T 435  ☑ 435 
Q 945-739 T 206  ☑ 206 
Q 71-39   T 32   ☑ 32  
Testing Accuracy :  1.0


### 1.2 實驗使用不同的epoch和batch size訓練
* with batch = 256, epoch = 100

In [17]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 256
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [18]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 5s 115us/step - loss: 1.8447 - acc: 0.3371 - val_loss: 1.6954 - val_acc: 0.3621

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 85us/step - loss: 1.6345 - acc: 0.3965 - val_loss: 1.5836 - val_acc: 0.4291

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 85us/step - loss: 1.5189 - acc: 0.4484 - val_loss: 1.4555 - val_acc: 0.4559

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 85us/step - loss: 1.4135 - acc: 0.4710 - val_loss: 1.3838 - val_acc: 0.4861A: 1

45000/45000 [==============================] - 4s 83us/step - loss: 0.0290 - acc: 0.9956 - val_loss: 0.0326 - val_acc: 0.9939

--------------------------------------------------
Iteration 34
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 83us/step - loss: 0.0265 - acc: 0.9962 - val_loss: 0.0306 - val_acc: 0.9935

--------------------------------------------------
Iteration 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 82us/step - loss: 0.0251 - acc: 0.9963 - val_loss: 0.0285 - val_acc: 0.9950

--------------------------------------------------
Iteration 36
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 81us/step - loss: 0.0233 - acc: 0.9968 - val_loss: 0.0268 - val_acc: 0.9945

--------------------------------------------------
Iteration 37
Train on 45000 samples, validate on 5000 samples
Epoch 1/1


45000/45000 [==============================] - 4s 83us/step - loss: 0.0053 - acc: 0.9997 - val_loss: 0.0100 - val_acc: 0.9977

--------------------------------------------------
Iteration 67
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 82us/step - loss: 0.0520 - acc: 0.9862 - val_loss: 0.0104 - val_acc: 0.9980

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 82us/step - loss: 0.0060 - acc: 0.9997 - val_loss: 0.0078 - val_acc: 0.9989

--------------------------------------------------
Iteration 69
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 82us/step - loss: 0.0050 - acc: 0.9998 - val_loss: 0.0074 - val_acc: 0.9990

--------------------------------------------------
Iteration 70
Train on 45000 samples, validate on 5000 samples
Epoch 1/1


45000/45000 [==============================] - 4s 80us/step - loss: 0.0049 - acc: 0.9992 - val_loss: 0.0047 - val_acc: 0.9992
Q 882-633 T 249  ☑ 249 
Q 327-16  T 311  ☑ 311 
Q 986-7   T 979  ☑ 979 
Q 771-110 T 661  ☑ 661 
Q 633-317 T 316  ☑ 316 
Q 926-422 T 504  ☑ 504 
Q 286-91  T 195  ☑ 195 
Q 623-200 T 423  ☑ 423 
Q 887-673 T 214  ☑ 214 
Q 159-11  T 148  ☑ 148 


In [19]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 224-3   T 221  ☑ 221 
Q 982-266 T 716  ☑ 716 
Q 709-370 T 339  ☑ 339 
Q 577-6   T 571  ☑ 571 
Q 8-5     T 3    ☑ 3   
Q 48-7    T 41   ☑ 41  
Q 39-3    T 36   ☑ 36  
Q 441-6   T 435  ☑ 435 
Q 945-739 T 206  ☑ 206 
Q 71-39   T 32   ☑ 32  
Testing Accuracy :  0.998


### 1.3 實驗使用不同的epoch和batch size訓練
* with batch = 128, epoch = 200

In [20]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [21]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 205us/step - loss: 1.7576 - acc: 0.3602 - val_loss: 1.6014 - val_acc: 0.4244

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 171us/step - loss: 1.4995 - acc: 0.4490 - val_loss: 1.4220 - val_acc: 0.4660

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 172us/step - loss: 1.3598 - acc: 0.4937 - val_loss: 1.2915 - val_acc: 0.5199

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 170us/step - loss: 1.1630 - acc: 0.5851 - val_loss: 1.0741 - val_acc: 0.6260


45000/45000 [==============================] - 8s 169us/step - loss: 0.0100 - acc: 0.9987 - val_loss: 0.0343 - val_acc: 0.9906

--------------------------------------------------
Iteration 34
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 167us/step - loss: 0.0298 - acc: 0.9922 - val_loss: 0.0123 - val_acc: 0.9980

--------------------------------------------------
Iteration 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 167us/step - loss: 0.0081 - acc: 0.9992 - val_loss: 0.0121 - val_acc: 0.9979

--------------------------------------------------
Iteration 36
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 167us/step - loss: 0.0263 - acc: 0.9929 - val_loss: 0.0135 - val_acc: 0.9977

--------------------------------------------------
Iteration 37
Train on 45000 samples, validate on 5000 samples
Epoch 

45000/45000 [==============================] - 8s 168us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 0.9993

--------------------------------------------------
Iteration 67
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 166us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 0.9990

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 166us/step - loss: 0.0403 - acc: 0.9893 - val_loss: 0.0053 - val_acc: 0.9989

--------------------------------------------------
Iteration 69
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 0.9994

--------------------------------------------------
Iteration 70
Train on 45000 samples, validate on 5000 samples
Epoch 


--------------------------------------------------
Iteration 99
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 0.0013 - acc: 0.9999 - val_loss: 0.0029 - val_acc: 0.9994

--------------------------------------------------
Iteration 100
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 9.9368e-04 - acc: 0.9999 - val_loss: 0.0025 - val_acc: 0.9993

--------------------------------------------------
Iteration 101
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 5.4707e-04 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 0.9994

--------------------------------------------------
Iteration 102
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 4.8389e-04 - acc: 1.0000 - val_loss: 0.0022

45000/45000 [==============================] - 7s 163us/step - loss: 2.3747e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9997

--------------------------------------------------
Iteration 132
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 2.1624e-04 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9997

--------------------------------------------------
Iteration 133
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 1.9714e-04 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9998

--------------------------------------------------
Iteration 134
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 164us/step - loss: 1.8128e-04 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9997

--------------------------------------------------
Iteration 135
Train on 45000 samples, validate on

45000/45000 [==============================] - 7s 163us/step - loss: 6.3124e-05 - acc: 1.0000 - val_loss: 0.0380 - val_acc: 0.9919

--------------------------------------------------
Iteration 164
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 0.0532 - acc: 0.9877 - val_loss: 0.0029 - val_acc: 0.9993

--------------------------------------------------
Iteration 165
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 165us/step - loss: 7.1753e-04 - acc: 0.9999 - val_loss: 0.0016 - val_acc: 0.9996

--------------------------------------------------
Iteration 166
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 166us/step - loss: 3.3885e-04 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 0.9997

--------------------------------------------------
Iteration 167
Train on 45000 samples, validate on 500

45000/45000 [==============================] - 7s 163us/step - loss: 7.6881e-05 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9998

--------------------------------------------------
Iteration 196
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 165us/step - loss: 6.9814e-05 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9998

--------------------------------------------------
Iteration 197
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 167us/step - loss: 6.3478e-05 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9998

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 170us/step - loss: 5.7929e-05 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 0.9998

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on

In [22]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 224-3   T 221  ☑ 221 
Q 982-266 T 716  ☑ 716 
Q 709-370 T 339  ☑ 339 
Q 577-6   T 571  ☑ 571 
Q 8-5     T 3    ☑ 3   
Q 48-7    T 41   ☑ 41  
Q 39-3    T 36   ☑ 36  
Q 441-6   T 435  ☑ 435 
Q 945-739 T 206  ☑ 206 
Q 71-39   T 32   ☑ 32  
Testing Accuracy :  0.999


## 2. 實驗不同位數的數字
* The digits of input number = 4

In [23]:
# Generate different data

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 4
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789- '
ctable = CharacterTable(chars)
print(ctable)

questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 9, 12)
(45000, 5, 12)
Validation Data:
(5000, 9, 12)
(5000, 5, 12)


In [27]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 5, 128)            131584    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 5, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [28]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 245us/step - loss: 1.7039 - acc: 0.3919 - val_loss: 1.5803 - val_acc: 0.4436

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 207us/step - loss: 1.4630 - acc: 0.4705 - val_loss: 1.3778 - val_acc: 0.5004

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 1.2669 - acc: 0.5246 - val_loss: 1.1706 - val_acc: 0.5589

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 192us/step - loss: 1.1031 - acc: 0.5871 - val_loss: 1.0524 - val_acc: 0.6176

45000/45000 [==============================] - 9s 190us/step - loss: 0.0601 - acc: 0.9818 - val_loss: 0.0744 - val_acc: 0.9755

--------------------------------------------------
Iteration 34
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0628 - acc: 0.9803 - val_loss: 0.0715 - val_acc: 0.9764

--------------------------------------------------
Iteration 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 192us/step - loss: 0.0505 - acc: 0.9854 - val_loss: 0.0977 - val_acc: 0.9654

--------------------------------------------------
Iteration 36
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 0.0551 - acc: 0.9830 - val_loss: 0.0758 - val_acc: 0.9744

--------------------------------------------------
Iteration 37
Train on 45000 samples, validate on 5000 samples
Epoch 

45000/45000 [==============================] - 9s 191us/step - loss: 0.0089 - acc: 0.9989 - val_loss: 0.0240 - val_acc: 0.9924

--------------------------------------------------
Iteration 67
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0076 - acc: 0.9992 - val_loss: 0.0238 - val_acc: 0.9930

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 0.0111 - acc: 0.9978 - val_loss: 0.0803 - val_acc: 0.9749

--------------------------------------------------
Iteration 69
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 0.0477 - acc: 0.9845 - val_loss: 0.0356 - val_acc: 0.9884

--------------------------------------------------
Iteration 70
Train on 45000 samples, validate on 5000 samples
Epoch 

45000/45000 [==============================] - 9s 190us/step - loss: 0.0024 - acc: 0.9999 - val_loss: 0.0182 - val_acc: 0.9945

--------------------------------------------------
Iteration 100
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0183 - val_acc: 0.9945

--------------------------------------------------
Iteration 101
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0170 - val_acc: 0.9944

--------------------------------------------------
Iteration 102
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0489 - acc: 0.9865 - val_loss: 0.0708 - val_acc: 0.9775

--------------------------------------------------
Iteration 103
Train on 45000 samples, validate on 5000 samples
Ep

45000/45000 [==============================] - 9s 204us/step - loss: 0.0031 - acc: 0.9996 - val_loss: 0.0196 - val_acc: 0.9947

--------------------------------------------------
Iteration 133
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 202us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0163 - val_acc: 0.9952

--------------------------------------------------
Iteration 134
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 201us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0158 - val_acc: 0.9955

--------------------------------------------------
Iteration 135
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 204us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0153 - val_acc: 0.9957

--------------------------------------------------
Iteration 136
Train on 45000 samples, validate on 5000 samples
Ep

45000/45000 [==============================] - 9s 192us/step - loss: 0.0019 - acc: 0.9998 - val_loss: 0.0161 - val_acc: 0.9954

--------------------------------------------------
Iteration 165
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0016 - acc: 0.9998 - val_loss: 0.0150 - val_acc: 0.9955

--------------------------------------------------
Iteration 166
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 190us/step - loss: 0.0125 - acc: 0.9968 - val_loss: 0.1047 - val_acc: 0.9700

--------------------------------------------------
Iteration 167
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 194us/step - loss: 0.0123 - acc: 0.9965 - val_loss: 0.0182 - val_acc: 0.9942

--------------------------------------------------
Iteration 168
Train on 45000 samples, validate on 5000 samples
Ep

45000/45000 [==============================] - 9s 191us/step - loss: 2.3039e-04 - acc: 1.0000 - val_loss: 0.0151 - val_acc: 0.9958

--------------------------------------------------
Iteration 197
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0540 - acc: 0.9860 - val_loss: 0.0276 - val_acc: 0.9918

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 191us/step - loss: 0.0029 - acc: 0.9994 - val_loss: 0.0209 - val_acc: 0.9937

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 192us/step - loss: 8.8196e-04 - acc: 1.0000 - val_loss: 0.0173 - val_acc: 0.9950
Q 9662-6    T 9656  ☑ 9656 
Q 9808-6582 T 3226  ☑ 3226 
Q 2806-70   T 2736  ☑ 2736 
Q 87-14     T 73    ☑ 73 

In [29]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [30]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 96-90     T 6     ☑ 6    
Q 6426-61   T 6365  ☑ 6365 
Q 9631-823  T 8808  ☑ 8808 
Q 5398-77   T 5321  ☑ 5321 
Q 955-94    T 861   ☑ 861  
Q 5260-82   T 5178  ☑ 5178 
Q 86-5      T 81    ☑ 81   
Q 5427-2    T 5425  ☑ 5425 
Q 8091-3783 T 4308  ☑ 4308 
Q 1545-42   T 1503  ☑ 1503 
Testing Accuracy :  0.984


## 3. Add more number
* Add 3 number (3 digits) together

In [32]:
def Generate_OtherData(DIGITS, DATA_SIZE, MAXLEN):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b, c = f(), f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        if(a>=(b+c) and (a+b)>=c):
            key = tuple(sorted((a, b, c)))
            if key in seen:
                continue
            seen.add(key)
            # Pad the data with spaces such that it is always MAXLEN.
            q = '{}-{}-{}'.format(a, b, c)
            query = q + ' ' * (MAXLEN - len(q))
            ans = str(a - b - c)
            # Answers can be of maximum size DIGITS + 1.
            ans += ' ' * (DIGITS + 1 - len(ans))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
                # space used for padding.)
                query = query[::-1]
            questions.append(query)
            expected.append(ans)
    print('Total addition questions:', len(questions))
    
    return questions, expected

In [33]:
# Generate different data

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789- '
ctable = CharacterTable(chars)
print(ctable)

questions, expected = Generate_OtherData(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 11, 12)
(45000, 4, 12)
Validation Data:
(5000, 11, 12)
(5000, 4, 12)


In [34]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [35]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 258us/step - loss: 1.6894 - acc: 0.3865 - val_loss: 1.5637 - val_acc: 0.4083

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 214us/step - loss: 1.4999 - acc: 0.4506 - val_loss: 1.4204 - val_acc: 0.4865

--------------------------------------------------
Iteration 3
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 217us/step - loss: 1.3507 - acc: 0.5018 - val_loss: 1.2952 - val_acc: 0.5258

--------------------------------------------------
Iteration 4
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 223us/step - loss: 1.2211 - acc: 0.5592 - val_loss: 1.1429 - val_acc: 0.6

45000/45000 [==============================] - 9s 209us/step - loss: 0.5536 - acc: 0.7940 - val_loss: 0.6195 - val_acc: 0.7665

--------------------------------------------------
Iteration 34
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 207us/step - loss: 0.5402 - acc: 0.7999 - val_loss: 0.5365 - val_acc: 0.7991

--------------------------------------------------
Iteration 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 207us/step - loss: 0.5333 - acc: 0.8013 - val_loss: 0.5374 - val_acc: 0.7925

--------------------------------------------------
Iteration 36
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 209us/step - loss: 0.5162 - acc: 0.8092 - val_loss: 0.5301 - val_acc: 0.8002

--------------------------------------------------
Iteration 37
Train on 45000 samples, validate on 5000 samples
Epoch 

45000/45000 [==============================] - 9s 199us/step - loss: 0.2460 - acc: 0.9133 - val_loss: 0.2785 - val_acc: 0.9005

--------------------------------------------------
Iteration 67
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 200us/step - loss: 0.2415 - acc: 0.9148 - val_loss: 0.3044 - val_acc: 0.8873

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 201us/step - loss: 0.2302 - acc: 0.9202 - val_loss: 0.2517 - val_acc: 0.9103

--------------------------------------------------
Iteration 69
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - loss: 0.2163 - acc: 0.9256 - val_loss: 0.2486 - val_acc: 0.9111

--------------------------------------------------
Iteration 70
Train on 45000 samples, validate on 5000 samples
Epoch 

45000/45000 [==============================] - 9s 208us/step - loss: 0.0256 - acc: 0.9950 - val_loss: 0.0567 - val_acc: 0.9826

--------------------------------------------------
Iteration 100
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 209us/step - loss: 0.0621 - acc: 0.9808 - val_loss: 0.0737 - val_acc: 0.9759

--------------------------------------------------
Iteration 101
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 209us/step - loss: 0.0250 - acc: 0.9951 - val_loss: 0.0393 - val_acc: 0.9873

--------------------------------------------------
Iteration 102
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 210us/step - loss: 0.0260 - acc: 0.9942 - val_loss: 0.0906 - val_acc: 0.9682

--------------------------------------------------
Iteration 103
Train on 45000 samples, validate on 5000 samples
Ep


--------------------------------------------------
Iteration 132
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 213us/step - loss: 0.0558 - acc: 0.9825 - val_loss: 0.0453 - val_acc: 0.9847

--------------------------------------------------
Iteration 133
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 210us/step - loss: 0.0112 - acc: 0.9982 - val_loss: 0.0244 - val_acc: 0.9928

--------------------------------------------------
Iteration 134
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 214us/step - loss: 0.0078 - acc: 0.9993 - val_loss: 0.0218 - val_acc: 0.9935

--------------------------------------------------
Iteration 135
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 209us/step - loss: 0.0076 - acc: 0.9992 - val_loss: 0.0236 - val_ac

KeyboardInterrupt: 

In [ ]:
## Generate Testing Data
questions_test, expected_test = Generate_OtherData(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

In [ ]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

# 總結
## 在Subtraction_rnn中採用lstm實現seq2seq的減法器
## 實驗:
### 1. 採用四種不同的batch size和訓練epochs (兩個三位數相減)
###     * [ batch size = 128, epoch = 100 ] : Validation acc = 99.97% / Testing acc (1000 testing data) = 0.998
###     * [ batch size = 64,   epoch = 100 ] : Validation acc = 99.99% / Testing acc (1000 testing data) = 1.0
###     * [ batch size = 256, epoch = 100 ] : Validation acc = 99.95% / Testing acc (1000 testing data) = 0.998
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 99.98% / Testing acc (1000 testing data) = 0.999
### 2. 兩個四位數相減
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 99.60% / Testing acc (1000 testing data) = 0.984
### 3. 三個三位數相減
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 99.24% / Testing acc (1000 testing data) = 0.98 